In [45]:
import json
import pandas as pd
import numpy as np
data_path = '../included_data/gpt3.5_generated_training_filtered.json'


data = []
with open(data_path, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [46]:
# df = df.groupby('error_type')

In [47]:
len(df['error_type'].unique())

15

In [48]:
size = 5        # sample size
replace = True  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
# samples = df.groupby('error_type', as_index=False).apply(fn)

samples = df.groupby('error_type').apply(lambda x: x.sample(5)).reset_index(drop=True)

In [49]:
samples.head()

,idx,error,correct,correct_token,wrong_token,chatgpt_response,prompt,model,error_type
0,4644,"When I was returning from Syleth , I called th...","When I was returning from Syleth , I called th...",the,[NONE],{'explanation': 'the error in the erroneous se...,The next two lines contain an erroneous senten...,gpt-3.5-turbo,Article
1,592,"Whatever the choice is , it will leave the mar...","Whatever the choice is , it will leave a mark ...",a,the,{'explanation': 'the error is the use of 'the ...,The next two lines contain an erroneous senten...,gpt-3.5-turbo,Article
2,12266,"Also , because of the fact that the urbanizati...","Also , because of the fact that urbanization ...",[NONE],the,{'explanation': 'the phrase 'the fact that' is...,The next two lines contain an erroneous senten...,gpt-3.5-turbo,Article
3,11100,All my yoga lessons and massage sessions are c...,All my yoga lessons and massage sessions are c...,the,[NONE],{'explanation': 'there should not be a space b...,The next two lines contain an erroneous senten...,gpt-3.5-turbo,Article
4,8061,"Furthermore , the government are enhancing the...","Furthermore , the government are enhancing pu...",[NONE],the,{'explanation': 'the error is in subject-verb ...,The next two lines contain an erroneous senten...,gpt-3.5-turbo,Article


In [50]:
len(samples)

75

In [51]:
type(samples)

pandas.core.frame.DataFrame

In [52]:
samples.columns

Index(['idx', 'error', 'correct', 'correct_token', 'wrong_token',
       'chatgpt_response', 'prompt', 'model', 'error_type'],
      dtype='object')

In [56]:
samples.drop(columns=['idx', 'correct_token', 'wrong_token', 'prompt', 'model']).to_csv('sampled_data.csv', index=False)

In [55]:
with open('samples.txt', 'w') as f:
    for i,r in samples.iterrows():
        f.write('Error_type: '+r['error_type'] + '\n')
        f.write('Error: ' +r['error'] + '\n')
        f.write('Correct: ' +r['correct'] + '\n')
        f.write('Wrong word: ' +r['wrong_token'] + '\n')
        f.write('Correct word: ' +r['correct_token'] + '\n')
        f.write('Explanation: ' +r['chatgpt_response']['explanation'] + '\n')
        f.write('Ex1_error: ' +r['chatgpt_response']['example1_erroneous'] + '\n')
        f.write('Ex1_correct: ' +r['chatgpt_response']['example1_correct'] + '\n')
        f.write('Ex2_error: ' +r['chatgpt_response']['example2_erroneous'] + '\n')
        f.write('Ex2_correct: ' +r['chatgpt_response']['example2_correct'] + '\n')
        f.write('Ex3_error: ' +r['chatgpt_response']['example3_erroneous'] + '\n')
        f.write('Ex3_correct: ' +r['chatgpt_response']['example3_correct'] + '\n')


        f.write('\n' + '-' * 80 + '\n')

In [32]:
type(df['chatgpt_response'][0])

dict

In [41]:
################# CLEANING DATA ####################
df  = df.dropna()
keys = ['explanation', 'example1_erroneous', 'example1_correct', 'example2_erroneous', 'example2_correct', 'example3_erroneous', 'example3_correct']
cnt = 0
bad_format = 0
bad = []
drop = []
replace = []
for i,r in df.iterrows():
    if isinstance(r['chatgpt_response'],dict):
        if list(r['chatgpt_response'].keys()) != keys:
            cnt += 1
            bad.append(r['chatgpt_response'])
            replace.append(i)
    else:
        print(r)
        bad_format += 1
    if len(list(r['chatgpt_response'].keys())) != 7:
        print(r['chatgpt_response'].keys())
        bad_format += 1
        drop.append(i)
        
df = df.drop(drop)
for i in replace:
    cnt = 0
    new_dict = {}
    for k,v in df.at[i,'chatgpt_response'].items():
        new_dict[keys[cnt]] = v
        cnt += 1
    df.at[i,'chatgpt_response'] = new_dict

df.to_json('gpt3.5_generated_training_filtered.json',orient='records')


In [42]:
bad_format,cnt

(0, 0)

In [43]:
len(df)

15183